In [85]:
# Latitude values are measured relative to the equator and range from -90° at the South Pole to +90° at the North Pole. Longitude values are measured relative to the prime meridian. They range from -180° when traveling west to 180° when traveling east. If the prime meridian is at Greenwich, then Australia, which is south of the equator and east of Greenwich, has positive longitude values and negative latitude values.
# http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm
# citipy??

# X Name a city, feed to OWM
# X Peform weather check
# X Understand JSON set up
# X Identify Temp, Humid, Cloud, Wind Speed

# X Randomly generate 1 lat/long combo within repspective ranges
# X Test in CitiPy; get city name
# X Feed to OWM
# Pull stuff from JSON

# Randomly generate 500 lat/longs
# Kill dupes
# Citipy
# Print this!

# Feed list to OWM
# Loop for info
# Store

# Create graphs
# Done

#How to work with citipy:

lat = 20
>>> lon = 75
>>> city = citipy.nearest_city(lat, lon)
>>> print(city)
<citipy.citipy.City object at 0x109724f98>
>>> city.city_name
'khuldabad'
>>> city.country_code
'in'

Openweather: Please, always use your API keys as &APPID=key


lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

just a heads up, using randomly generated lats/longs can pick points in the ocean. citipy.nearest_city can be used to find cities that can be queried on

Processing Record 1 of Set 1 | longyearbyen

In [86]:
# Import Dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import requests
from citipy import citipy
from config import api_key

#api.openweathermap.org/data/2.5/weather?q=London,uk

target_url = "http://api.openweathermap.org/data/2.5/weather" \
    "?q=Phoenix,us&units=IMPERIAL&mode=json&APPID=" + api_key
    
#api.openweathermap.org/data/2.5/weather?q=London,uk

In [87]:
base_url = "http://api.openweathermap.org/data/2.5/weather"

params = {
    "q": "New York,US",
    "units": "IMPERIAL",
    "mode": "json",
    "APPID": api_key
}

In [88]:
response = requests.get(base_url, params=params)
print(response.url)

http://api.openweathermap.org/data/2.5/weather?q=New+York%2CUS&units=IMPERIAL&mode=json&APPID=84e9790cfed06e9f791a3c163ba4c987


In [89]:
city_weather = response.json()
city_weather.keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'id', 'name', 'cod'])

In [90]:
city_weather['coord']

{'lat': 40.71, 'lon': -74.01}

In [91]:
city_weather['coord']['lat']

40.71

In [92]:
temp = city_weather['main']['temp']
humidity = city_weather['main']['humidity']
wind_speed = city_weather['wind']['speed']
cloudiness = city_weather['clouds']['all']

#print(f'Processing Record 1 of Set 1 | {city}')
#print(temp, humidity, wind_speed, cloudiness)

In [93]:
#Processing Record 1 of Set 1 | longyearbyen

In [94]:
#lats = np.random.uniform(low=-90.000, high=90.000, size=1500) 
#lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

one_lat = np.random.uniform(low=-90.000, high=90.000) 
one_long = np.random.uniform(low=-180.000, high=180.000)

print(f'Lat: {one_lat}, Long: {one_long}')

Lat: 22.441247278391117, Long: 113.43125303305857


In [95]:
# lat = 20
# >>> lon = 75
# >>> city = citipy.nearest_city(lat, lon)
# >>> print(city)
# <citipy.citipy.City object at 0x109724f98>
# >>> city.city_name
# 'khuldabad'
# >>> city.country_code
# 'in'

one_city = citipy.nearest_city(one_lat, one_long)
#print(one_city)
city_name = one_city.city_name
city_country_code = one_city.country_code

print(city_name, city_country_code)


zhongshan cn


In [96]:
query = city_name + ',' + city_country_code
print(query)

zhongshan,cn


In [97]:
base_url = "http://api.openweathermap.org/data/2.5/weather"

params = {
    "q": query,
    "units": "IMPERIAL",
    "mode": "json",
    "APPID": api_key
}
response = requests.get(base_url, params=params)
print(response.url)

http://api.openweathermap.org/data/2.5/weather?q=zhongshan%2Ccn&units=IMPERIAL&mode=json&APPID=84e9790cfed06e9f791a3c163ba4c987
